# Set up OpenID APIs
Inspect an OpenID connect well-known JSON for the variables you need elsewhere


In [ ]:
# Enter the .well-known URL for the auth service you want to use
WELL_KNOWN_URL = input('Enter .well-known/openid-configuration URL: ') or 'https://accounts.google.com/.well-known/openid-configuration'

In [ ]:
import json
import requests
from urllib.parse import urlparse

def get_json(url):
    resp = requests.get(url)
    try:
        resp.raise_for_status()
    except Exception as ex:
        print(resp.content)
        raise ex
    return resp.json()

url_info = urlparse(WELL_KNOWN_URL)
DOMAIN, _, _ = url_info.netloc.partition(':') # removes the port 
print(f"Looking up details for {DOMAIN}")

SSL_KEY_PATH = "" #"./local/TPP_OB_Transport.key"
SSL_CERT_PATH = "" #"./local/TTP_OB_Transport.pem"

WELL_KNOWN = get_json(WELL_KNOWN_URL)
print(json.dumps(WELL_KNOWN, indent=2))


In [ ]:
AUTH_METHODS = WELL_KNOWN.get('token_endpoint_auth_methods_supported', [])
AUTHORIZATION_URL = WELL_KNOWN.get('authorization_endpoint')
TOKEN_URL = WELL_KNOWN.get('token_endpoint')
REGISTER_URL = WELL_KNOWN.get('registration_endpoint')
ISSUER = WELL_KNOWN.get('issuer')

print(f"Issuer: {ISSUER}")
print(f"Token endpoint auth methods: {', '.join(AUTH_METHODS)}")
print(f"AUTHORIZATION_URL: {AUTHORIZATION_URL}")
print(f"TOKEN_URL: {TOKEN_URL}")
print(f"REGISTER_URL: {REGISTER_URL}")

# Create APIs for Dynamic Registration

In [ ]:
# Enter intended project's APImetrics API key here:
API_KEY = input("Enter your APImetrics API key for the intended project: ")
SOFTWARE_STATEMENT = input("Enter the Software statement you wish to use: ") or "{{software_statement}}"

In [ ]:
# Helper functions
import requests
import json
import urllib
from apimetrics_api import APImetricsAPI 
    
# An instance of the class that calls the APImetrics API
CLIENT = APImetricsAPI(API_KEY)

In [ ]:
# First, create APImetrics API Auth Setting
tag = 'auth:apimetrics_api'
if tag not in CLIENT.auths_by_tag:
    setup = {
        "meta": {
            "domain": "client.apimetrics.io",
            "documentation": {
                "keys": "https://client.apimetrics.io/settings/api-key",
                "docs": "https://apimetrics.readme.io/v2/reference",
                "apps": "",
                "provider": "https://client.apimetrics.io/",
            },
            "name": "APImetrics API",
            "tags": [tag],
            "description": "API that allows you to call APImetrics' API.",
        },
        "settings": {"auth_type": "MANUAL"}
    }
    auth = CLIENT.create_auth(setup)
    print(f"Created Auth Setting {auth['meta']['name']} with id {auth['id']}")
else:
    print(f"Found Auth Setting {CLIENT.auths_by_tag[tag]}")

# Next, Create Token
if CLIENT.auths_by_tag[tag] not in CLIENT.tokens_by_auth:
    setup = {
        'meta': {
            'name': 'Project Access Token',
            'domain': 'client.apimetrics.io',
            'auth_id': CLIENT.auths_by_tag[tag]
        },
        'token': {
            'headers': [
              {
                'p_key': 'Authorization',
                'p_val': f'Bearer {API_KEY}',
              },
            ],
        }
    }
    token = CLIENT.create_token(setup)
    print(f"Created Auth Token {token['meta']['name']} with id {token['id']}")

In [ ]:
# Create Auth Setting for Bank API
tag = 'auth:bank_matls'
if tag not in CLIENT.auths_by_tag:

    ssl_key = None
    ssl_cert = None

    if SSL_KEY_PATH:
        with open(SSL_KEY_PATH) as stream:
            ssl_key = stream.read()

    if SSL_CERT_PATH:
        with open(SSL_CERT_PATH) as stream:
            ssl_cert = stream.read()
    
    setup = {
        "access": {
            "keys": False,
            "org_keys": False,
            "org_settings": True,
            "settings": False,
        },
        "keys": {},
        "meta": {
            "domain": DOMAIN,
            "documentation": {"keys": "", "docs": "", "apps": "", "provider": ""},
            "name": "Transport MATLS",
            "tags": [tag],
            "description": "Mutual Authenticated TLS for calls to bank APIs",
        },
        "settings": {
            "auth_type": "MANUAL",
            "ssl_key": ssl_key,
            "ssl_cert": ssl_cert,
        },
    }
#     if OAUTH_METHOD == "client_secret_post" or OAUTH_METHOD == "client_secret_basic":
#         setup["keys"]["client_id"] = CLIENT_ID
#         setup["keys"]["client_secret"] = CLIENT_SECRET
    
    auth = CLIENT.create_auth(setup)
    print(f"Created Auth Setting {auth['meta']['name']} with id {auth['id']}")

In [ ]:
# For user authentication
tag = 'jwt:sign:dynamic_registration'
if tag not in CLIENT.calls_by_tag:
    auth_tag = 'auth:apimetrics_api'
    auth_id = CLIENT.auths_by_tag[auth_tag]
    token_id = CLIENT.tokens_by_auth[auth_id]
    
    body = {
        "header": {},
        "payload": {
            "scopes": [
              "openid",
              "accounts",
              "payments"
            ],
            "software_statement": SOFTWARE_STATEMENT,
            "redirect_uris": [
              "{{redirect_uri}}",
              "https://client.apimetrics.io/tokens/callback/"
            ],
            "grant_types": [
              "authorization_code",
              "refresh_token",
              "client_credentials"
            ],
            "response_types": [
              "code id_token"
            ],
            "token_endpoint_auth_method": "private_key_jwt",
            "token_endpoint_auth_signing_alg": "RS256",
            "id_token_signed_response_alg": "RS256",
            "request_object_signing_alg": "RS256",
            "request_object_encryption_alg": "RSA-OAEP-256",
            "request_object_encryption_enc": "A128CBC-HS256"
        },
        "private_key": {
            "versionId": '{{kid_cert_version}}'
        }
    }
    
    body_str = json.dumps(body, indent=2)
    
    setup = {
        "meta": {
            "tags": ["api_type:create", "sector:devtools", tag],
            "name": "Sign Dynamic Client Registration JWT with KMS Cert",
            "workspace": "global",
        },
        "request": {
            "body": body_str,
            "parameters": [
                {"value": "{{kid}}", "key": "kid"},
                {"value": "RS256", "key": "alg"},
                {"key": "noTimestamp", "value": "1"},
                {"value": "{{ssa_id}}", "key": "iss"},
                {"value": "{{aud}}", "key": "aud"},
                {"value": "10m", "key": "expiresIn"},
            ],
            "url": "https://us-central1-viatests.cloudfunctions.net/jwt-kms-signer",
            "auth_id": auth_id,
            "headers": [
                {"key": "Accept", "value": "application/json"},
                {"key": "Content-Type", "value": "application/json"},
            ],
            "token_id": token_id,
            "method": "POST",
        },
    }
    call = CLIENT.create_call(setup)
    print(f"Created Call {call['meta']['name']} with id {call['id']}")

In [ ]:
tag = 'banks:3.1:dynamic_registration:create'
if tag not in CLIENT.calls_by_tag:
    setup = {
      "meta": {
        "description": None, 
        "tags": [
            "api_type:create", 
            "sector:financial",
            tag,
        ], 
        "name": "Dynamic Client Registration: Create", 
        "workspace": "global"
      }, 
      "request": {
        "body": "__JWT_TOKEN__", 
        "parameters": [], 
        "url": REGISTER_URL, 
        "auth_id": CLIENT.auths_by_tag['auth:bank_matls'], 
        "headers": [
          {
            "value": "application/jwt", 
            "key": "Content-Type"
          }
        ], 
        "token_id": None, 
        "method": "POST"
      }
    }
    call = CLIENT.create_call(setup)
    print(f"Created Call {call['meta']['name']} with id {call['id']}")

In [ ]:
tag = 'banks:3.1:dynamic_registration'
if tag not in CLIENT.workflows_by_tag:
    call_tags = [
        'jwt:sign:dynamic_registration',
        'banks:3.1:dynamic_registration:create',
    ]
    for t in call_tags:
        assert t in CLIENT.calls_by_tag, f"API {t} does not exist"

    setup = {
      "meta": {
        "name": "Dynamic Client Registration", 
        "workspace": "global", 
        "tags": [tag], 
      }, 
      "workflow": {
        "handle_cookies": False,
        "stop_on_failure": True,
        "call_ids": [CLIENT.calls_by_tag[t] for t in call_tags]
      }
    }
    workflow = CLIENT.create_workflow(setup)
    print(f"Created Workflow {workflow['meta']['name']} with id {workflow['id']}")

## Set Environment Variables

In [ ]:
CLIENT.set_env_variable("global", "ssa_id", input("Software Statement ID: ") or "mTxvBFQd99jOaqgn7TgAr2")

In [ ]:
CLIENT.set_env_variable("global", "software_statement", input("Software Statement: ") )

In [ ]:
CLIENT.set_env_variable("global", "kid", input("Key ID: ") or "oGZO56CED_eru44IGcVzA2tk8NA" )

In [ ]:
CLIENT.set_env_variable("global", "kid_cert_version", input("KMS cert version: "))

In [ ]:
CLIENT.set_env_variable("global", "aud", ISSUER)

In [ ]:
CLIENT.set_env_variable("global", "redirect_uri", "https://google.api.expert/get")